In [1]:
#%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import matplotlib.cm     as cm
import matplotlib.collections as mc
import numpy as np
import cv2

from larlite import larlite as fmwk
from ROOT import fcllite

cfg_mgr = fcllite.ConfigManager()
cfg_mgr.AddCfgFile("../mac/ClusterHoughPSet.fcl")

config = cfg_mgr.Config()
pset = config.get_pset("ClusterHough"); 

my_proc = fmwk.ana_processor()
my_proc.add_input_file("/Users/vgenty/Desktop/larlite_reco2d_000.root")

my_proc.set_io_mode(fmwk.storage_manager.kBOTH)                                                                                                                     
my_proc.set_output_file("aho.root")
my_proc.set_data_to_write(fmwk.data.kHit,'gaushit')                                                                                                                 
my_proc.set_data_to_write(fmwk.data.kCluster,'hullcluster')                                                                                                         
my_proc.set_data_to_write(fmwk.data.kAssociation,'hullcluster')                 

my_unit  = fmwk.HitImageManager()
my_aho   = fmwk.AlgoImageMaker()
my_aho2  = fmwk.AlgoClusterConvex(pset)

my_unit.SetImageMaker(my_aho)
my_unit.SetClusterMaker(my_aho2)
my_proc.add_process(my_unit)    

my_proc.process_event()
my_proc.process_event()

True

In [2]:
my_proc.process_event()

True

In [ ]:
plt.imshow(my_aho2.GetDilatedImage(2),cmap = cm.Greys,interpolation='none')
plt.show()

In [3]:
plt.imshow(my_aho2.GetBinaryImage(2),cmap = cm.Greys,interpolation='none')
plt.show()

/Library/Python/2.7/site-packages/matplotlib/image.py:676: UserWarning: The backend (<class 'matplotlib.backends.backend_macosx.RendererMac'>) does not support interpolation='none'. The image will be interpolated with 'nearest` mode.
  "mode." % renderer.__class__)


In [2]:
fig, ax = plt.subplots(figsize=(10,6))


#houghs = my_aho2.houghs(2)
#hh=[]
#for h in xrange(houghs.size()):
#    l = houghs[h]
#    hh.append([(l[0],l[1]),(l[2],l[3])])
#cc = mc.LineCollection(hh, linewidths=1)

hulls = my_aho2.hulls(2)
for x in xrange(hulls.size()):
    hull = hulls[x]
    
    x_array = []
    y_array = []
    for p in xrange(hull.size()):
        x_array.append(hull[p].first)
        y_array.append(hull[p].second)
    x_array.append(x_array[0])
    y_array.append(y_array[0])  
    
    ax.plot(x_array,y_array,color='black',lw=2)
 
pts = my_aho2.other_hits(2)
xx= []
yy= []
for x in xrange(pts.size()):
    p = pts[x]

    xx.append(p.first)
    yy.append(p.second)
    
ax.plot(xx,yy,'o',markersize=1)

pclusters = my_aho2.p_clusters(2)

xx= []
yy= []
for x in xrange(pclusters.size()):
    pcluster = pclusters[x]
    
    hits = pcluster.hits()
    xx= []
    yy= []
    
    for i in xrange(hits.size()):
        hit = hits[i]
        xx.append(hit.first)
        yy.append(hit.second)
    

    ax.plot(xx,yy,'o',markersize=3,c=np.random.rand(3,1))
    
#ax.add_collection(cc)

plt.show()

In [10]:
print my_aho2.possible_break_num(2)

3


In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
plane = 2
idx   = 1
ax.imshow(my_aho2.possible_break(plane,idx),cmap = cm.Greys,interpolation='none')

houghs = my_aho2.real_hough_v(plane)[idx]
hh=[]
for h in xrange(houghs.size()):
    l = houghs[h]
    hh.append([(l[0],l[1]),(l[2],l[3])])

cc = mc.LineCollection(hh, linewidths=1)
ax.add_collection(cc)
plt.tight_layout()
plt.show()

In [15]:
plane = 2
idx   = 1

r = my_aho2.possible_break(plane,idx)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
r = cv2.dilate(r,kernel,iterations = 2)
r = cv2.blur(r,(5,5))
ret,r = cv2.threshold(r,0,255,cv2.THRESH_BINARY)

In [16]:
_, contours0, hierarchy = cv2.findContours( r.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = [cv2.approxPolyDP(cnt, 1, True) for cnt in contours0]
len(contours)

1

In [17]:
x1  = []
y1  = []
xy1 = []

for i in contours[0]:
    x1.append(i[0][0])
    y1.append(i[0][1])
    xy1.append( [i[0][0], i[0][1]] )

x1.append(x1[0])
y1.append(y1[0])
xy1.append([x1[0], y1[0]])

chull = cv2.convexHull(np.array(xy1),returnPoints = False)
chull = chull[:,0].tolist()
chull.append(chull[0])
defects = cv2.convexityDefects(np.array(xy1),np.array(chull))

x2 = []
y2 = []
for i in chull:
    x2.append(x1[i])
    y2.append(y1[i])

In [18]:
fig,ax = plt.subplots(figsize=(10,6))

ax.plot(x1,y1,color='blue',lw=2) 
ax.plot(x2,y2,color='red',lw=2) 

lines = []
px = []
py = []

for i in xrange(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = (x1[s],y1[s])
    end =   (x1[e],y1[e])
    far =   (x1[f],y1[f])
    lines.append([start, far])
    lines.append([far, end])
    px.append(x1[f])
    py.append(y1[f])

lc = mc.LineCollection(lines, linewidths=10,color='green',alpha=0.7)
ax.imshow(r,cmap = cm.Greys,interpolation='none')
ax.add_collection(lc)
plt.plot(px,py,'o',color='purple')
ax.autoscale()
plt.show()

In [19]:
for i in xrange(len(defects)):
    print defects[i][0][3]/256.0

1.6640625
3.4921875


In [ ]:
chull.tolist() + [ chull[0][0] ]

In [ ]:
k = chull.tolist()

In [ ]:
k.append(list(10))

In [ ]:
print chull